In [1]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self, match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.batting_first = self.get_batting_first()
        self.batting_second = self.get_batting_second()
        # self.team = self.get_team()
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.team1_dismissals_df = self.decider1()
        self.team2_dismissals_df = self.decider2()
        self.dream11_points = self.get_dream11_points()

        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    # def get_team(self, team):
    #     team = []
    #     for i in self.json_api['rosters'][0]['roster']:
    #          team.append(i['athlete']['name'])
    #     return team
    
    def get_batting_first(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return teamName
            else:
                return otherTeamName[0]
        else:
            if(inningsNumber == '2'):
                return teamName
            else:
                return otherTeamName[0]

    def get_batting_second(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return otherTeamName[0]
            else:
                return teamName
        else:
            if(inningsNumber == '2'):
                return otherTeamName[0]
            else:
                return teamName

    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_team1_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 0
        else:
            z = 1

        # if self.home_team['name'] == self.batting_second:
        #     a = 1
        # else:
        #     a = 0

        for i in self.json_api['rosters'][0]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    
    def get_team2_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 1
        else:
            z = 0

        # if self.home_team['name'] == self.batting_first:
        #     a = 1
        # else:
        #     a = 0 

        for i in self.json_api['rosters'][1]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    def decider1(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team1_dismissals_df()
        else:
            return self.get_team2_dismissals_df()
        
    def decider2(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team2_dismissals_df()
        else:
            return self.get_team1_dismissals_df()

In [2]:
matchNumber = 1426296
# t = Match(1426295)
m = Match(matchNumber)


/tmp/ipykernel_3450/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_3450/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_3450/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [3]:
match = Match(1426294)

/tmp/ipykernel_3450/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_3450/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_3450/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [4]:
team1batting = match.team1_batting_df
team2batting = match.team2_batting_df
team1dismiss = match.team1_dismissals_df
team2dismiss = match.team2_dismissals_df
team1bowling = match.team1_bowling_df
team2bowling = match.team2_bowling_df

In [5]:
team1batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Jake Fraser-McGurk,c Ferreira b Ashwin,50,20,7,3,250.00
1,Abishek Porel,c Sandeep Sharma b Ashwin,65,36,7,3,180.56
2,Shai Hope,run out (Sandeep Sharma),1,1,0,0,100.00
3,Axar Patel,c Parag b Ashwin,15,10,1,1,150.00
4,Rishabh Pant,c Boult b Chahal,15,13,0,1,115.38
5,Tristan Stubbs,lbw b Sandeep Sharma,41,20,3,3,205.00
6,Gulbadin Naib,c Ashwin b Boult,19,15,1,1,126.67
7,Rasikh Salam,run out (Dubey/†Samson),9,3,2,0,300.00
8,Kuldeep Yadav,not out,5,2,1,0,250.00


In [6]:
team1dismiss
# problem = if home team batting first den this is wrong

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Jake Fraser-McGurk,Batter,c,Ravichandran Ashwin,Donovan Ferreira
1,Abishek Porel,Wicketkeeper batter,c,Ravichandran Ashwin,Sandeep Sharma
2,Shai Hope,Wicketkeeper batter,run out,None,Sandeep Sharma
3,Axar Patel,Bowling allrounder,c,Ravichandran Ashwin,Riyan Parag
4,Rishabh Pant,Wicketkeeper batter,c,Yuzvendra Chahal,Trent Boult
5,Tristan Stubbs,Batter,lbw,Sandeep Sharma,None
6,Gulbadin Naib,Batting allrounder,c,Trent Boult,Ravichandran Ashwin
7,Rasikh Salam,Bowler,run out,None,Shubham Dubey/Sanju Samson
8,Kuldeep Yadav,Bowler,not out,None,None
9,Mukesh Kumar,Bowler,0,None,None


In [7]:
team2batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Yashasvi Jaiswal,c Patel b Ahmed,4,2,1,0,200.00
1,Jos Buttler,b Patel,19,17,2,1,111.76
2,Sanju Samson,c Hope b Mukesh Kumar,86,46,8,6,186.96
3,Riyan Parag,b Rasikh Salam,27,22,1,3,122.73
4,Shubham Dubey,c Stubbs b Ahmed,25,12,2,2,208.33
5,Rovman Powell,b Mukesh Kumar,13,10,1,1,130.00
6,Donovan Ferreira,lbw b Kuldeep Yadav,1,3,0,0,33.33
7,Ravichandran Ashwin,c Hope b Kuldeep Yadav,2,3,0,0,66.67
8,Trent Boult,not out,2,3,0,0,66.67
9,Avesh Khan,not out,7,3,1,0,233.33


In [8]:
team2dismiss
# Problem vice versa

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Yashasvi Jaiswal,Opening batter,c,Khaleel Ahmed,Axar Patel
1,Jos Buttler,Wicketkeeper batter,0,Axar Patel,None
2,Sanju Samson,Wicketkeeper batter,c,Mukesh Kumar,Shai Hope
3,Riyan Parag,Top-order batter,0,Rasikh Salam,None
4,Shubham Dubey,Middle-order batter,c,Khaleel Ahmed,Tristan Stubbs
5,Rovman Powell,Middle-order batter,0,Mukesh Kumar,None
6,Donovan Ferreira,Allrounder,lbw,Kuldeep Yadav,None
7,Ravichandran Ashwin,Bowling allrounder,c,Kuldeep Yadav,Shai Hope
8,Trent Boult,Bowler,not out,None,None
9,Avesh Khan,Bowler,not out,None,None


In [9]:
team1bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Khaleel Ahmed,4.0,0,47,2,11.75,9,5,2,7,0
2,Ishant Sharma,3.0,0,34,0,11.33,7,4,2,1,0
3,Mukesh Kumar,3.0,0,30,2,10.00,7,4,1,1,0
5,Axar Patel,3.0,0,25,1,8.33,9,2,2,0,0
7,Kuldeep Yadav,4.0,0,25,2,6.25,10,0,2,0,1
9,Rasikh Salam,3.0,0,36,1,12.00,6,1,4,1,0


In [10]:
team2bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Trent Boult,4.0,0,48,1,12.0,8,7,2,0,0
2,Sandeep Sharma,4.0,0,42,1,10.5,7,5,2,0,0
4,Avesh Khan,2.0,0,42,0,21.0,0,5,3,0,0
5,Ravichandran Ashwin,4.0,0,24,3,6.0,6,0,1,0,0
7,Riyan Parag,2.0,0,17,0,8.5,4,1,1,1,0
8,Yuzvendra Chahal,4.0,0,48,1,12.0,5,4,3,0,0


In [11]:
def generate_batting_points(x, y):
    x = x.merge(y)
    x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x.drop(x.iloc[:, 1:11], inplace=True, axis=1)
    return x

def generate_bowling_points(y):
    y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])
    y.drop(y.iloc[:, 1:11], inplace=True, axis=1)
    y.rename(columns = {'BOWLING':'Name'}, inplace = True)
    return y

# def generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling):
#     team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
#     team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
#     # adding catching points
#     return pd.concat([team1points, team2points]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

team1batting = generate_batting_points(team1batting, team1dismiss)
team2batting = generate_batting_points(team2batting, team2dismiss)
team1bowling = generate_bowling_points(team1bowling)
team2bowling = generate_bowling_points(team2bowling)
# generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling)

In [63]:
def generate_points(team1batting, team2batting, team1bowling, team2bowling, team2dismiss, team1dismiss):
    a = generate_points1(team1batting, team2dismiss, team1bowling)
    b = generate_points2(team2batting, team1dismiss, team2bowling)
    return pd.concat([a, b]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)

def generate_points1(team1batting,team2dismiss, team1bowling):
    team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return fielding1(team1points, team2dismiss)

def generate_points2(team1batting,team2dismiss, team1bowling):
    team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return fielding2(team1points, team2dismiss)

def fielding1(team1points,team2dismiss):
    catcher = team2dismiss.loc[team2dismiss['Dismissal Type'] == 'c']
    catcher = catcher['Catcher/Runout']
    catcher = dict(zip(*numpy.unique(catcher, return_counts=True)))
    output = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in catcher:
        output.loc[len(output)] = [i, catcher[i] * 8]
    runout = team2dismiss.loc[team2dismiss['Dismissal Type'] == 'run out']
    runout = runout['Catcher/Runout']
    l = {}
    for i in runout:
        if len(i.split('/')) > 1:
            l[i.split('/')[0]] = 6
            l[i.split('/')[1]] = 6
        else:
            l[i] = 12
    output2 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in l:
        output2.loc[len(output2)] = [i, l[i]]
    pd.concat([team1points, output]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return pd.concat([team1points, output2]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)

def fielding2(team2points,team1dismiss):
    catcher = team1dismiss.loc[team1dismiss['Dismissal Type'] == 'c']
    catcher = catcher['Catcher/Runout']
    catcher = dict(zip(*numpy.unique(catcher, return_counts=True)))
    output = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in catcher:
        output.loc[len(output)] = [i, catcher[i] * 8]
    runout = team1dismiss.loc[team1dismiss['Dismissal Type'] == 'run out']
    runout = runout['Catcher/Runout']
    l = {}
    for i in runout:
        if len(i.split('/')) > 1:
            l[i.split('/')[0]] = 6
            l[i.split('/')[1]] = 6
        else:
            l[i] = 12
    output2 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in l:
        output2.loc[len(output2)] = [i, l[i]]
    pd.concat([team2points, output]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return pd.concat([team2points, output2]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)


generate_points(team1batting, team2batting, team1bowling, team2bowling, team2dismiss, team1dismiss)

,Name,Fantasy Points
17,Sanju Samson,120
0,Abishek Porel,86
12,Ravichandran Ashwin,79
6,Jake Fraser-McGurk,71
9,Kuldeep Yadav,56
21,Tristan Stubbs,50
8,Khaleel Ahmed,50
10,Mukesh Kumar,50
2,Axar Patel,43
19,Shubham Dubey,37


In [13]:
team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)

In [14]:
team1points

,Name,Fantasy Points
0,Abishek Porel,86
4,Jake Fraser-McGurk,71
6,Kuldeep Yadav,56
5,Khaleel Ahmed,50
11,Tristan Stubbs,50
7,Mukesh Kumar,50
1,Axar Patel,43
8,Rasikh Salam,36
2,Gulbadin Naib,22
9,Rishabh Pant,17


In [15]:
team2dismiss

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Yashasvi Jaiswal,Opening batter,c,Khaleel Ahmed,Axar Patel
1,Jos Buttler,Wicketkeeper batter,0,Axar Patel,None
2,Sanju Samson,Wicketkeeper batter,c,Mukesh Kumar,Shai Hope
3,Riyan Parag,Top-order batter,0,Rasikh Salam,None
4,Shubham Dubey,Middle-order batter,c,Khaleel Ahmed,Tristan Stubbs
5,Rovman Powell,Middle-order batter,0,Mukesh Kumar,None
6,Donovan Ferreira,Allrounder,lbw,Kuldeep Yadav,None
7,Ravichandran Ashwin,Bowling allrounder,c,Kuldeep Yadav,Shai Hope
8,Trent Boult,Bowler,not out,None,None
9,Avesh Khan,Bowler,not out,None,None


In [16]:
team1points

,Name,Fantasy Points
0,Abishek Porel,86
4,Jake Fraser-McGurk,71
6,Kuldeep Yadav,56
5,Khaleel Ahmed,50
11,Tristan Stubbs,50
7,Mukesh Kumar,50
1,Axar Patel,43
8,Rasikh Salam,36
2,Gulbadin Naib,22
9,Rishabh Pant,17


In [17]:
team1bowling

,Name,Fantasy Points
0,Khaleel Ahmed,50
2,Ishant Sharma,0
3,Mukesh Kumar,50
5,Axar Patel,25
7,Kuldeep Yadav,50
9,Rasikh Salam,25


In [18]:
# pd.merge(team1batting , team1dismiss)

team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
# team1batting.merge(team2bowling,on=['Name'], how='outer').groupby(['Fantasy Points'], as_index=False)['count'].sum()
# team1bowling.groupby(['Name'],as_index=False).agg({'Fantasy Points': 'sum'})

In [19]:
team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

In [20]:
pd.concat([team1points, team2points]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

,Name,Fantasy Points
17,Sanju Samson,114
0,Abishek Porel,86
12,Ravichandran Ashwin,79
6,Jake Fraser-McGurk,71
9,Kuldeep Yadav,56
10,Mukesh Kumar,50
8,Khaleel Ahmed,50
21,Tristan Stubbs,50
2,Axar Patel,43
11,Rasikh Salam,36


In [21]:
team1batting

,Name,Fantasy Points
0,Jake Fraser-McGurk,71
1,Abishek Porel,86
2,Shai Hope,1
3,Axar Patel,18
4,Rishabh Pant,17
5,Tristan Stubbs,50
6,Gulbadin Naib,22
7,Rasikh Salam,11
8,Kuldeep Yadav,6


In [22]:
team1bowling

,Name,Fantasy Points
0,Khaleel Ahmed,50
2,Ishant Sharma,0
3,Mukesh Kumar,50
5,Axar Patel,25
7,Kuldeep Yadav,50
9,Rasikh Salam,25


In [23]:
team2batting

,Name,Fantasy Points
0,Yashasvi Jaiswal,5
1,Jos Buttler,23
2,Sanju Samson,114
3,Riyan Parag,34
4,Shubham Dubey,31
5,Rovman Powell,16
6,Donovan Ferreira,1
7,Ravichandran Ashwin,2
8,Trent Boult,2
9,Avesh Khan,8


In [24]:
team2bowling

,Name,Fantasy Points
0,Trent Boult,25
2,Sandeep Sharma,25
4,Avesh Khan,0
5,Ravichandran Ashwin,77
7,Riyan Parag,0
8,Yuzvendra Chahal,25
